**<center>BỘ GIÁO DỤC VÀ ĐÀO TẠO</center>**
**<center>TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT TP.HỒ CHÍ MINH</center>**
**<center>KHOA CÔNG NGHỆ THÔNG TIN</center>**
**<center>BÁO CÁO ĐỒ ÁN</center>**
**<center>MÔN HỌC : KHOÁ LUẬN TỐT NGHIỆP</center>**
**<center>TÊN ĐỀ TÀI: DIFFERENTIAL PRIVACY IN DEEP LEARNING</center>**
**<center>GVHD: PGS.TS Hoàng Văn Dũng</center>**
**<center>GVPB: ThS Lê Thị Minh Châu</center>**

**Sinh viên thực hiện**<br>
- Huỳnh Nguyễn Như Nguyên - 20133019<br>
- Lê Minh Trí - 20133100<br>

### Giới thiệu về notebook
Notebook này mô tả về giả định quá trình tấn công suy luận thành viên dựa trên nguyên lý tấn công hộp đen (black-box) vào mô hình mục tiêu khi không có sự bảo vệ của Differential Privacy.


#### 1. Tải và tiền xử lý dữ liệu

In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset,  random_split
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, fake_dir, real_dir, transform=None):
        self.fake_dir = fake_dir
        self.real_dir = real_dir
        self.transform = transform

        self.fake_images = [(os.path.join(fake_dir, img), 1) for img in os.listdir(fake_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]
        self.real_images = [(os.path.join(real_dir, img), 0) for img in os.listdir(real_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

        self.all_images = self.fake_images + self.real_images

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        img_path, label = self.all_images[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# Đường dẫn tới thư mục chứa ảnh
fake_dir = './data/fake'
real_dir = './data/real'
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Tăng cường màu sắc
    transforms.ToTensor(),
])
# Khởi tạo dataset và dataloader
dataset = CustomDataset(fake_dir, real_dir, transform=transform)


#### 2. Định nghĩa cấu trúc mô hình

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512 * 4 * 4, 512)  
        self.fc2 = nn.Linear(512, 2)  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.pool(F.relu(self.conv6(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### 3. Chia tập dữ liệu

In [3]:
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 2331
Validation size: 499
Test size: 500


#### 4. Lấy ảnh và nhãn tương ứng từ tập huấn luyện và tập kiểm tra

In [4]:
images = []
labels = []

for i in range(len(test_dataset)):
    image, label = test_dataset[i]
    images.append(image)
    labels.append(label)
images_tensor = torch.stack(images)
labels_tensor = torch.tensor(labels)

In [5]:
imagesTrain = []
labelsTrain = []

for i in range(len(train_dataset)//2+1):
    image, label = train_dataset[i]
    imagesTrain.append(image)
    labelsTrain.append(label)
images_tensorTrain = torch.stack(imagesTrain)
labels_tensorTrain = torch.tensor(labelsTrain)

In [6]:
labels_tensorTrain

tensor([1, 1, 1,  ..., 1, 1, 0])

#### 5. Thực hiện tấn công suy luận thành viên 

In [7]:
import PIL.Image
from art.estimators.classification import  PyTorchClassifier
import numpy as np
model = CNNModel()
model.load_state_dict(torch.load('./model/FakeAndRealState.pth'))
model.eval()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=1e-5)
Classifier= PyTorchClassifier(model,criterion,[3,256,256],2)
from art.attacks.inference.membership_inference import MembershipInferenceBlackBoxRuleBased
attack = MembershipInferenceBlackBoxRuleBased(Classifier)
inferred_train = attack.infer(images_tensorTrain, labels_tensorTrain)
inferred_test = attack.infer(images_tensor, labels_tensor)
train_acc = np.sum(inferred_train) / len(inferred_train)
test_acc = 1 - (np.sum(inferred_test) / len(inferred_test))
acc = (train_acc * len(inferred_train) + test_acc * len(inferred_test)) / (len(inferred_train) + len(inferred_test))
print(f"Members Accuracy: {train_acc:.4f}")
print(f"Non Members Accuracy {test_acc:.4f}")
print(f"Attack Accuracy {acc:.4f}")




Members Accuracy: 0.7590
Non Members Accuracy 0.2440
Attack Accuracy 0.6044


#### 6. Thử nghiệm tấn công trên mục tiêu đơn lẻ

In [8]:
import PIL
import cv2
image=PIL.Image.open('./data/testattack.jpg')
image = np.array(image) 
image = cv2.resize(image, (256, 256))
image = image.astype(np.float32) / 255.0
image = image.transpose(2, 0, 1) 
image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  
label=np.array([0])
label=torch.tensor(label)
inferred_t=attack.infer(image,label)
print('thành viên training' if inferred_t == 1 else 'Không phải thành viên')

thành viên training
